<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_1002T5_demo_filling_blank_question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 「はごろも」をもちいた T5 による文章穴埋め問題作成の試み
- datafile: `hagoromo-data-20180918.xlsx` 
- date: 2022_1002
- author: 浅川伸一
- filename: `2022_1002T5_demo_filling_blank_question.ipynb`


In [ ]:
%config InlineBackend.figure_format = 'retina'
try:
    import bit
except ImportError:
    !pip install --upgrade 'fugashi[ipadic]' > /dev/null 2>&1
    !pip install --upgrade 'fugashi[unidic]' > /dev/null 2>&1
    !python -m unidic download
    !pip install --upgrade ipadic > /dev/null 2>&1
    !pip install --upgrade sentencepiece > /dev/null 2>&1
    !pip install --upgrade transformers > /dev/null 2>&1

    !pip install 'konoha[all]' > /dev/null 2>&1
    !pip install --upgrade termcolor > /dev/null 2>&1
    !pip install --upgrade jaconv  > /dev/null 2>&1      
    !pip install ipynbname --upgrade > /dev/null 2>&1
    !git clone https://github.com/ShinAsakawa/bit.git
    import bit

isColab = bit.isColab
HOME = bit.HOME

In [ ]:
if isColab:
    from google.colab import files
    uploaded = files.upload()  # ここで `hagoromo-data-20180918.xlsx` を指定してアップロード
    data_dir = '.'
else:
    data_dir = os.path.join(HOME, 'study/2022jlpt')


In [ ]:
import pandas as pd
import jaconv  
hagoromo_fname = 'hagoromo-data-20180918.xlsx'
hag = pd.read_excel(hagoromo_fname)
hag

In [ ]:
from termcolor import colored
import jaconv

from konoha import SentenceTokenizer
splitter = SentenceTokenizer()

_hag = hag['例文'].to_list()

hag_sentences = []
max_len = 0
min_len = 30000
for i, l in enumerate(_hag):
    if isinstance(l, str):
        
        _l = jaconv.normalize(l)
        
        # hagoromo データには '/' と '／' とが混在して用いられているようだ
        # もしかしたら意味があるのかも知れないが，現時点では問い合わせていない。
        # そのためデータ前処理として '/' を '／' に置換して文の区切りとして用いる
        _l = _l.replace('/','／') 
        
        #_l = _l.replace('\n','')
        
        # 上と同じく ',' と '、' とが混在して用いられているようなので '、' に統一
        _l = _l.replace(',','、')  
        
        _l2 = _l.split('／') 
        _l3_ = []
        for _l4 in _l2:
            for _l5 in splitter.tokenize(_l4):
                _l3_.append(_l5)
            
        # 一つのエクセルセル内に複数の文が登録されていので分割
        #_l2 = splitter.tokenize(_l)
        for _l3 in _l3_:
            
            len_l = len(_l3)
            
            if len_l >= 1:
                hag_sentences.append(_l3)
                if len_l > max_len:
                    max_len = len_l
                    print(f'line:{i:5d}:{_l3} max_len:{max_len}')
                if len_l < min_len:
                    min_len = len_l
                    print(f'line:{i:5d}:{_l3} min_len:{min_len}')
    elif isinstance(l, float):
        continue
    else:
        print(colored(f'({i},{l},{type(l)}',"red",attrs=['bold']))

In [ ]:
for x in hag_sentences[:5]:
    print(x)

In [ ]:
import torch
from transformers import T5Tokenizer, RobertaForMaskedLM

# load tokenizer
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-roberta-base")
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

# load model
model = RobertaForMaskedLM.from_pretrained("rinna/japanese-roberta-base")
model = model.eval()


## 使用上の注意

* `[CLS]` を使用すること。`[CLS]` はモデルの学習時に使用されるため，マスクされたトークンを予測するには，必ず文の前に `[CLS]` トークンを追加して，モデルが正しく符号化できるようにする。
* トークン化した後に `[MASK]` を使用する。入力文字列に直接 [MASK] を入力するのと，トークン化した後に `[MASK]` で置き換えるのとでは、トークンの並びが異なるので，予測結果も異なる。
トークン化後に `[MASK]` を使用する方がよい。
これは，モデルがどのように事前学習されたかと一致するためである。
しかし Huggingface Inference API は入力文字列に `[MASK]` をタイプすることしかサポートしておらず，よりロバストな予測を生成することができない。
* `position_ids` を明示的に引数として与える。
`Roberta` モデルでは，モデルに対して `position_ids` が与えられない場合 Huggingface 版の transformers は自動的に `position_ids` を構築する。だが， 0 ではなく `padding_idx` から作成する。これでは `rinna/japanese-roberta-base` では期待通りに動作しない。
なぜなら対応するトークナイザーの `padding_idx` が 0 ではないからである。
そのため，必ず自分で `position_ids` を `constrcut` して，位置 ID 0 から開始するようにすること。

In [ ]:
def gpt_predict(
    full_text:str, 
    blank:str,
    top_k:int=10,
    model=model,
    ):
    model = model.eval()                
    
    text = '[CLS]' + full_text
    tokens = tokenizer.tokenize(text)
    
    # convert to ids
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    
    masked_tokens = tokenizer.tokenize(blank)[1:]
    masked_pos = []
    for masked_token in masked_tokens:
        #print(f'masked_token:{masked_token}')
        try:
            if tokens.index(masked_token):
                pos = tokens.index(masked_token)
        except ValueError:
            pos = -1
        if pos != -1:
            token_ids[pos] = tokenizer.mask_token_id
            masked_pos.append(pos)

    print(f'トークン ID をもう一度トークンに戻す:{tokenizer.convert_ids_to_tokens(token_ids)}')
            
    # convert to tensor
    token_tensor = torch.LongTensor([token_ids])

    position_ids = list(range(0, token_tensor.size(1)))
    position_id_tensor = torch.LongTensor([position_ids])
    
    # マスクに対応する上位 top_k 候補を得る
    with torch.no_grad():
        outputs = model(input_ids=token_tensor, 
                        position_ids=position_id_tensor)
    for _masked_pos in masked_pos:
        predictions = outputs[0][0, _masked_pos].topk(10)
        print(f'_masked_pos:{_masked_pos}')
        for i, index_t in enumerate(predictions.indices):
            index = index_t.item()
            token = tokenizer.convert_ids_to_tokens([index])[0]
            print(f'\t{i}:{token}', end=" ")
        print()                            
    #print(tokens)

gpt_predict(
    full_text='このパソコンは誰でも使えますが，コピーは有料です', 
    blank='有料',
    model=model)

gpt_predict(
    full_text='次々に新しいゲームが作られる。', 
    blank='次々に',
    model=model)
